## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

## Initialization

In [183]:
import numpy as np # For numerical calculations
import cv2 # For OpenCV libraries
import glob # For group input of images
import matplotlib.pyplot as plt # Plotting library
%matplotlib qt

# For video IO and processing
from moviepy.editor import VideoFileClip
from IPython.display import HTML

## 1. Camera Calibration using Chessboard Images

In [184]:
def cameraCalibration():
    # Using checkerboard images with 9x6 number of inside indices
    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d points in real world space
    imgpoints = [] # 2d points in image plane.

    # Make a list of calibration images
    images = glob.glob('camera_cal/calibration*.jpg')

    # Step through the list and search for chessboard corners
    for fname in images:
        img = cv2.imread(fname) # Reads each frame as an iamge
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) # Convert t ograyscale
        img_size = (gray.shape[1], gray.shape[0])

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)

    # Use all found corners and image points
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

    return mtx, dist

## 2. Distortion Correction

In [185]:
def undistort(img, mtx, dist):
    # Use generated parameters to undistort each image without recalculating coefficients
    undistorted = cv2.undistort(img, mtx, dist, None, mtx)

    return undistorted

## 3. Color/Gradient Thresholds with Gaussian Blur

In [186]:
def colorGradient(img, s_thresh=(170, 255), sx_thresh=(21, 100)):
    copy = np.copy(img)
    
    #Use kernal size to filter out noise
    kernel_size = 11
    blur = cv2.GaussianBlur(copy, (kernel_size, kernel_size), 0)
    
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(blur, cv2.COLOR_RGB2HLS)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]

    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold color channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1

    # Stack each channel
    color_binary = np.zeros_like(s_channel)
    color_binary[(s_binary == 1) | (sxbinary == 1)] = 1
    
    return color_binary

## 4. Perspective Transform (Bird's Eye View)

In [187]:
# Define a function that takes an image, number of x and y points, 
# camera matrix and distortion coefficients
def perspectiveTransform(img):
    # Estimated source and destination values for a bird's eye view of the road
    # Values were determined using estimates from two straight line images
    h = img.shape[0]
    w = img.shape[1]
    img_size = (w,h)
    mid_offset = 90

    # Top left, top right, bottom left, bottom right
    src = np.float32([[w/2-mid_offset, 460], [w/2+mid_offset, 460], [0, h-15], [w, h-15]])
    dst = np.float32([[0, 0], [w, 0], [0, h], [w, h]])

    # Given src and dst points, calculate the perspective transform matrix
    M = cv2.getPerspectiveTransform(src, dst)

    # Warp the image using OpenCV warpPerspective()
    transformed = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)

    # Return the resulting image and matrix
    return transformed

## 5. Lane Boundary Detection and Vehicle Position

In [188]:
def detectLane(image, left_fit, right_fit):
    # Check if this is the first iteration -> Use windows and histograms
    if (left_fit == [0,0,0] and right_fit == [0,0,0]):
        # Find our lane pixels first
        leftx, lefty, rightx, righty = windowHistograms(image)
    else:
        leftx, lefty, rightx, righty = searchAroundPoly(image, left_fit, right_fit)
    
    # Obtain polynomial coefficients in both pixels and meters
    left_fitx, right_fitx, left_fit_rw, right_fit_rw, ploty = fit_poly(image.shape, leftx, lefty, rightx, righty)
    
    # Calculate curvature in meters
    curvature = measure_curvature_real(ploty, left_fit_rw, right_fit_rw)
    
    # Calculate offset in meters using the meters per pixel ratio
    # Assumption: Center of the image/video is the center of the car
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    offset = (640 - (left_fitx[-1] + right_fitx[-1])/2)*xm_per_pix
    
    # Visualization using generated polynomial lane curve estimates
    margin = 30 # Pixels on each side of the polynomial curve
    out_img = np.dstack((image, image, image))*255
    
    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    # Left Lane
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                              ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    # Right Lane
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                              ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))
    # Central (Safe) Area
    center_window1 = np.array([np.transpose(np.vstack([left_fitx+margin, ploty]))])
    center_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx-margin, 
                              ploty])))])
    center_pts = np.hstack((center_window1, center_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(out_img, np.int_([left_line_pts]), (255, 0, 0))
    cv2.fillPoly(out_img, np.int_([right_line_pts]), (0, 0, 255))
    cv2.fillPoly(out_img, np.int_([center_pts]), (0, 255, 0))

    return out_img, left_fit, right_fit, curvature, offset

In [189]:
def fit_poly(img_shape, leftx, lefty, rightx, righty):
    # Fit a second order polynomial for both the left and right lane data points
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
   
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty
    
    # For the real-world polyfit
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension
    left_fit_rw = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    right_fit_rw = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    
    return left_fitx, right_fitx, left_fit_rw, right_fit_rw, ploty

In [190]:
def windowHistograms(image):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(image[image.shape[0]//2:,:], axis=0)

    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Hyperparamters
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 80
    # Set minimum number of pixels found to recenter window
    minpix = 30

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(image.shape[0]//nwindows)
    
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = image.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = image.shape[0] - (window+1)*window_height
        win_y_high = image.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin

        # Identify non-zero pixels that are within the window bounds for each window
        good_left_inds = ((nonzeroy < win_y_high) & (nonzeroy >= win_y_low) & 
        (nonzerox < win_xleft_high) & (nonzerox >= win_xleft_low)).nonzero()[0]
        good_right_inds = ((nonzeroy < win_y_high) & (nonzeroy >= win_y_low) *
        (nonzerox < win_xright_high) & (nonzerox >= win_xright_low)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)

        # Change the next rectangle's center location if the number of detected pixels
        # exceed the minimum threshold
        if (len(good_left_inds) > minpix):
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if (len(good_right_inds) > minpix):
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    return leftx, lefty, rightx, righty

In [191]:
def searchAroundPoly(image, left_fit, right_fit):
    # Hyperparamter - width of the search +_polynomial curve
    margin = 60

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])

    # Calculate indices that fit within the margins of the polynomial curve
    left_margins = left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2]
    right_margins = right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2]
    left_lane_inds = ((nonzerox < left_margins + margin) & (nonzerox > left_margins
    - margin))
    right_lane_inds = ((nonzerox < right_margins + margin) & (nonzerox > right_margins
    - margin))

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    return leftx, lefty, rightx, righty

## 6. Lane Curvature

In [192]:
def measure_curvature_real(ploty, left_fit, right_fit):
    # Using the left/right fit in meters, calculate the curvature at the bottom of the image
    y_eval = np.max(ploty)
    
    # Using the curve as f(y) = Ay^2 + By + C:
    # The curvature can be estimated as: (1+(2Ay+B)^2)^(3/2)/|2A|
    left_curverad = ((1 + (2*left_fit[0]*y_eval + left_fit[1])**2)**1.5) / np.absolute(2*left_fit[0])
    right_curverad = ((1 + (2*right_fit[0]*y_eval + right_fit[1])**2)**1.5) / np.absolute(2*right_fit[0])
    
    return np.int((left_curverad + right_curverad)//2)

## 7. Inverse Perspective Transform and Text Overlay

In [193]:
def inversePerspectiveTransform(img):
    # Estimated source and destination values for a bird's eye view of the road
    # Values were determined using estimates from two straight line images
    h = img.shape[0]
    w = img.shape[1]
    img_size = (w,h)
    mid_offset = 90
    
    # Top left, top right, bottom left, bottom right
    src = np.float32([[w/2-mid_offset, 460], [w/2+mid_offset, 460], [0, h-15], [w, h-15]])
    dst = np.float32([[0, 0], [w, 0], [0, h], [w, h]])

    # Given src and dst points, calculate the perspective transform matrix
    Minv = cv2.getPerspectiveTransform(dst, src)

    # Warp the image using OpenCV warpPerspective()
    transformed = cv2.warpPerspective(img, Minv, img_size, flags=cv2.INTER_LINEAR)

    # Return the resulting image and matrix
    return transformed

In [194]:
def addText(img, curvature, offset):
    # Font and sizes is a default CV2 font
    font = cv2.FONT_HERSHEY_SIMPLEX
    
    # Positioned to be in the top left corner with 50px margins
    bottomLeftCornerOfText1 = (50,50)
    bottomLeftCornerOfText2 = (50,100)
    fontScale = 1
    
    # White color 
    fontColor = (255,255,255)
    lineType = 2
    
    # Depending on the vehicle offset, different words are used to maintain a positive value shown
    if (offset == 0):
        text1 = f"Radius of Curvature = {curvature}m"
        text2 = "Vehicle is centered"
    elif (offset < 0):
        offset = np.abs(offset)
        text1 = f"Radius of Curvature = {curvature}m"
        text2 = f"Vehicle is {offset:.2}m left of center"
    else:
        text1 = f"Radius of Curvature = {curvature}m"
        text2 = f"Vehicle is {offset:.2}m right of center"
        
    # Text is placed onto the image and returned
    cv2.putText(img, text1, bottomLeftCornerOfText1, font, fontScale, fontColor, lineType)
    cv2.putText(img, text2, bottomLeftCornerOfText2, font, fontScale, fontColor, lineType)

## Full Image Processing

In [195]:
def process_image(img):
    # Uses the left and right fits
    global left_fit, right_fit
    
    # Calibrate the camera to fix distotions using checkerboard patterns taken from the same camera
    mtx, dist = cameraCalibration()
    calibrated = undistort(img, mtx, dist)
    
    # Pipeline for color and gradient thresholding for a binary image of detected pixels
    binary = colorGradient(calibrated)
    
    # Perspective transform binary pixels into a bird's eye view
    transformed = perspectiveTransform(binary)
    
    # Lane is detected using window sections of polynomial margins, depending on if it's the first iteration
    curve, left_fit, right_fit, curvature, offset = detectLane(transformed, left_fit, right_fit)
    
    # Inverse transform the image back to the original shape
    inverse = inversePerspectiveTransform(curve)
    
    # Add the text for curvature and car position offset    
    addText(img, curvature, offset)
    
    # The detected lane markings are overlaid with a 50% opacity before returning
    result = cv2.addWeighted(img, 1, inverse, 0.5, 0)

    return result

In [196]:
# Testing Purposes only
left_fit = [0,0,0]
right_fit = [0,0,0]
img = cv2.imread('test_images/test5.jpg')
out = process_image(img)
cv2.imwrite('report_images/final_image.jpg', out)
cv2.imshow('image', out)

### Applied to a Video

left_fit = [0,0,0]
right_fit = [0,0,0]
video_output = 'project_video_output.mp4'

clip = VideoFileClip("project_video.mp4")
video_clip = clip.fl_image(process_image)
%time video_clip.write_videofile(video_output, audio=False)

HTML("""
<video width="1280" height="720" controls>
  <source src="{0}">
</video>
""".format(video_output))